**Author:** A.S. Grm (aleksander.grm@fpp.uni-lj.si)

**Date:** 2024

<hr>

# Calculation of Height Intercept and Azimuth for Celestial Body (SRP - Sight Reduction Procedure)

The calculation of positional parameters using the *Mark De Saint Hilaire* method is carried out manually using the [Nautical Almanac](https://www.thenauticalalmanac.com) (*Gha*, *Dec*, and *Sha* of Aries and the celestial body) along with the measurement of the celestial body's altitude $H$.

In addition to the measurement itself, we need to determine the measurement time and aproximate location. Positions are entered in the format commonly used in navigation:

$\varphi = \#\#^\circ \, \#\#.\#' \, \mathrm{N/S}$, $\lambda = \#\#\#^\circ \, \#\#.\#' \, \mathrm{E/W}$,

where degrees are given as whole numbers and minutes to one decimal place precision.

Input format for position:
- LAT-$\varphi$ and LONG-$\lambda$ are entered in the format [degrees, minutes, cardinal direction]

Example input for a position:
- $\varphi$: [12, 34.5, 'N']
- $\lambda$: [117, 12.6, 'W']
- The cardinal direction must be in **single quotes**, with permissible characters 'N', 'S', 'E', 'W'

Measurement time is recorded as:
- date: *date*=[day,month,year]
- time: *time*=[hour,minutes, seconds]

The format for entering the height is the same as for the position and is written as:
- height: $h=\#\#^\circ \#\#.\#'$=[deg,min] (example: [34, 17.6])

<hr>

**Calculation Procedure**

The procedure for determining the positional parameters proceeds in the following order:

1. First, we need to select an assumed position (AP):
   $$P_\mathrm{AP} = (\varphi_\mathrm{AP},\lambda_\mathrm{AP}),$$

2. Next, calculate the height of the observed celestial body (calculated height $H_c$) using the formula:
   $$ \sin H_c = \sin \varphi_\mathrm{AP} \: \sin \delta + \cos \varphi_\mathrm{AP} \: \cos \delta \: \cos \mathrm{Lha},$$

3. Then, calculate the azimuth of the celestial body from our assumed position:
   $$\cos \omega = \frac{\sin \delta - \sin H_c \: \sin \varphi_\mathrm{AP}}{\cos H_c \: \cos \varphi_\mathrm{AP}}$$
   and convert it into a navigational azimuth required for plotting the line of position:
   $$\omega_\mathrm{N} = \begin{cases} \omega &; \text{if } 180^\circ < \mathrm{Lha} \leq 360^\circ \\ 360^\circ - \omega &; \text{if } 0^\circ < \mathrm{Lha}  \leq 180^\circ \end{cases}$$
   This converts the calculated azimuth $\omega$ into a navigational azimuth $\omega_\mathrm{N}$ using the local hour angle $\mathrm{Lha}$.

4. With the help of the measured and corrected height $H_\mathrm{o}$ (observed height), we can determine the height difference, known as the **intercept** in English:
   $$\Delta H [\mathrm{Nm}]= 60 \: (H_\mathrm{o}[\mathrm{deg}] - H_\mathrm{c}[\mathrm{deg}]).$$
   The difference is determined in nautical miles (Nm) because we need a distance to plot the line of position on the chart.

<hr>

## Calculation program 

In [ ]:
import os, sys

# add custom modules and astro data path 
pp = '../nav_tools/'
sys.path.append(pp)

In [ ]:
import math as mat
import numpy as np
import matplotlib.pyplot as mpl
mpl.rcParams['text.usetex'] = True
mpl.rcParams.update({'font.size': 7})

import celestialdata as cdata
import navigationalstars as ns
import navtools as nt

In [ ]:
def getHeight(fi, dec, lha):

    fi_rad = nt.deg2rad(fi)
    dec_rad = nt.deg2rad(dec)
    lha_rad = nt.deg2rad(lha)

    sin_hc = mat.sin(fi_rad)*mat.sin(dec_rad) + mat.cos(fi_rad)*mat.cos(dec_rad)*mat.cos(lha_rad)

    return mat.asin(sin_hc) 

In [ ]:
def getAzimuth(fi, dec, lha, hc):

    fi_rad = nt.deg2rad(fi)
    dec_rad = nt.deg2rad(dec)
    lha_rad = nt.deg2rad(lha)

    cos_wc = (mat.sin(dec_rad) - mat.sin(hc)*mat.sin(fi_rad))/(mat.cos(hc)*mat.cos(fi_rad))

    wc = mat.acos(cos_wc)

    if lha > mat.pi:
        wn = wc
    else:
        wn = 2*mat.pi - wc

    return wn

In [ ]:
# Finds all celestial bodies above the horizon
def findAllVisibleCelestialBodies(t,pos,ns_db):
    
    star_ids = ns_db.keys()
    h0 = 0
    h1 = 90
    
    vs = []
    for s_id in star_ids:
        sdd = cd.get_star_data(s_id,t)
        saz = cd.get_star_altaz(s_id,t,pos)
        if saz['alt'] > h0 and saz['alt'] < h1:
            s_name = ns_db[s_id][0]
            vs.append({'name':s_name, 'dec':sdd['dec'], 'sha':sdd['sha'], 'alt':saz['alt'], 'az':saz['az']})
    
    return vs

In [ ]:
# Finds CB data
def findCelestialBodyData(ns_db,pos,t,name):
 
    gha_a = cd.get_aries_gha(t)
    lha_a = gha_a + pos[1]

    cbs = findAllVisibleCelestialBodies(t,pos,ns_db)
    dec = 0
    sha = 0
    alt = 0
    az = 0
    found = False

    for cb in cbs:
        if cb['name'].upper() == name.upper():
            dec = cb['dec']
            sha = cb['sha']
            alt = cb['alt']
            az = cb['az']
            found = True

    data = {'gha_a':gha_a, 'lha_a':lha_a, 'dec':dec, 'sha':sha, 'alt':alt, 'az':az}

    return [found,data,cbs]   

In [ ]:
#
def getHeightAndAzimuth(ns_db,pos,date,time,name):

    t = [date[0],date[1],date[2],time[0],time[1],time[2]]

    [cf, data,cbs] = findCelestialBodyData(ns_db,pos,t,name)

    if cf:
        lha = data['lha_a'] + data['sha']
        hc = nt.rad2deg(getHeight(pos[0], data['dec'], lha))
        wn = nt.rad2deg(getAzimuth(pos[0], data['dec'], hc, lha))

        return [cf,hc,wn,cbs]
    else:
        return [cf,0,0,cbs] 

<hr>

## Input data


In [ ]:
# Load astro database
cd = cdata.CelestialData(pp)
ns_db = cd.get_nav_stars_db()

In [ ]:
# *** Inputs ***
date = [2022,2,12] # [yyyy, mm, dd]
time = [19,0,0]    # [HH:MM:SS] in UTC

# Apparent position
fi = [46,0,'N']; 
la = [13,30,'E']

# Observer height in [meters]
h = 20.0

# Observed Celestial Body 
cb_name = 'betelgeuse' # Celestial Body name
ho = [51, 11.28]       # Celestial Body observed height [deg, mi.dec] (corrected!!)


# -------------------
# --- calculation ---
# -------------------

pos = [nt.nav2dd(fi), nt.nav2dd(la), h]
ho_dd = ho[0]+ho[1]/60

[cf, hc, wn, cbs] = getHeightAndAzimuth(ns_db,pos,date,time,cb_name)

if cf:
    dh = 60 * (ho_dd - hc)
    print('Results of calculation:')
    print()
    print('  -> body: {:s}'.format(cb_name.upper()))
    print('  -> height calculated: {:s}'.format(nt.prettyPrintAlt(hc)))
    print('  -> height measured:   {:s}'.format(nt.prettyPrintAlt(ho_dd)))
    
    print()
    print('  -> azimuth:   {:s}'.format(nt.prettyPrintAz(wn)))
    print('  -> intercept: {:6.2f} Nm'.format(dh))
else:
    print('  ERROR: Celestial body name {:s} NOT found in database'.format(cb_name))
    print('         Visible bodies are:')
    for cb in cbs:
        print('           -> {:12s}'.format(cb['name']))
        